<a href="https://colab.research.google.com/github/candidowe/DIO_Tradutor/blob/main/traduto_dio_artigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4 openai langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.77
    Uninstalling langchain-core-0.3.77:
      Successfully uninstalled langchain-core-0.3.77


In [2]:
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from langchain_openai.chat_models.azure import AzureChatOpenAI

# --- Carrega as variáveis de ambiente do config.env ---
load_dotenv("config.env")

# Lê as variáveis do arquivo .env
AZURE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
AZURE_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

# --- Função para extrair texto limpo de uma URL ---
def extract_text_from_url(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        for script_or_style in soup(["script", "style"]):
            script_or_style.decompose()

        text = soup.get_text(separator=' ')
        linhas = (line.strip() for line in text.splitlines())
        part = (phrase.strip() for line in linhas for phrase in line.split("  "))
        texto_limpo = '\n'.join(part for part in part if part)
        return texto_limpo
    else:
        print(f"Falha ao acessar a página. Código: {response.status_code}")
        return None


# --- Inicializa o cliente Azure OpenAI ---
client = AzureChatOpenAI(
    azure_endpoint=AZURE_ENDPOINT,
    api_key=AZURE_KEY,
    api_version=AZURE_VERSION,
    deployment_name=AZURE_DEPLOYMENT,
    max_retries=0
)

# --- Função de tradução ---
def translate_article(text, lang):
    messages = [
        ("system", "Você é um tradutor de textos técnicos e artigos."),
        ("user", f"Traduza o seguinte texto para o idioma {lang} e mantenha a formatação em Markdown:\n\n{text}")
    ]
    response = client.invoke(messages)
    print(response.content)
    return response.content


# --- Exemplo de uso ---
if __name__ == "__main__":
    url = "https://dev.to/edgaras/ethereum-integration-in-php-for-wallets-and-transfers-11d2"
    article_text = extract_text_from_url(url)

    if article_text:
        translated = translate_article(article_text[:8000], "português")

        # Opcional: salvar tradução em arquivo .md
        with open("artigo_traduzido.md", "w", encoding="utf-8") as f:
            f.write(translated)
        print("\n✅ Tradução salva em 'artigo_traduzido.md'")


OpenAIError: Missing credentials. Please pass one of `api_key`, `azure_ad_token`, `azure_ad_token_provider`, or the `AZURE_OPENAI_API_KEY` or `AZURE_OPENAI_AD_TOKEN` environment variables.